In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
from datetime import datetime as dt

In [6]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            ts = dt.now().isoformat()
            print(f"[{ts}] Processing row {i}")
            print()
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [8]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

[2019-02-07T10:43:58.790681] Processing row 0

[2019-02-07T10:47:35.572179] Processing row 50

[2019-02-07T10:52:45.720879] Processing row 100

[2019-02-07T10:56:13.755235] Processing row 150

[2019-02-07T10:59:39.087885] Processing row 200

[2019-02-07T11:02:49.487927] Processing row 250

[2019-02-07T11:06:25.505190] Processing row 300

[2019-02-07T11:09:26.628359] Processing row 350

[2019-02-07T11:12:58.089931] Processing row 400

[2019-02-07T11:14:59.467262] Processing row 450

[2019-02-07T11:19:06.482728] Processing row 500

[2019-02-07T11:22:15.612210] Processing row 550

[2019-02-07T11:25:37.887490] Processing row 600

[2019-02-07T11:29:57.342770] Processing row 650

[2019-02-07T11:34:37.794535] Processing row 700

[2019-02-07T11:38:15.379254] Processing row 750

[2019-02-07T11:41:25.042996] Processing row 800

[2019-02-07T11:44:11.903865] Processing row 850

[2019-02-07T11:46:45.536138] Processing row 900

[2019-02-07T11:49:29.599243] Processing row 950

[2019-02-07T11:53:20.18

,dreamy,warning,convenient,alteration,idiotic,ready,waiting,two,satisfied,tie,...,nearly,charge,symptom,since,custom,rational,earnest,snug,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [9]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

c:\users\kylem\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Training set score: 0.987460815047022

Test set score: 0.8984962406015038


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [11]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8824451410658307

Test set score: 0.8707706766917294


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [12]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [13]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [14]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [15]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

[2019-02-07T18:19:03.137720] Processing row 0

[2019-02-07T18:20:07.265973] Processing row 50

[2019-02-07T18:20:44.910725] Processing row 100

[2019-02-07T18:21:31.623956] Processing row 150

[2019-02-07T18:22:39.455965] Processing row 200

[2019-02-07T18:24:00.755665] Processing row 250

[2019-02-07T18:25:16.862893] Processing row 300

[2019-02-07T18:26:05.772631] Processing row 350

[2019-02-07T18:26:54.646403] Processing row 400

[2019-02-07T18:27:28.036121] Processing row 450

[2019-02-07T18:28:16.598915] Processing row 500

[2019-02-07T18:28:54.455636] Processing row 550

[2019-02-07T18:29:41.279597] Processing row 600

[2019-02-07T18:30:25.025708] Processing row 650

[2019-02-07T18:31:12.395545] Processing row 700

[2019-02-07T18:31:54.267177] Processing row 750

[2019-02-07T18:32:35.139452] Processing row 800

[2019-02-07T18:33:14.505358] Processing row 850

[2019-02-07T18:33:58.179360] Processing row 900

[2019-02-07T18:34:42.046888] Processing row 950

[2019-02-07T18:35:21.17

In [16]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

### Challenge 0:

In [17]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.6824451410658308

Test set score: 0.6917293233082706


In [18]:
word_counts.head()

,dreamy,warning,convenient,alteration,idiotic,ready,waiting,two,satisfied,tie,...,nearly,charge,symptom,since,custom,rational,earnest,snug,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [19]:
len(word_counts.text_sentence[0])

67

In [20]:
word_counts['num_words'] = [len(text) for text in word_counts.text_sentence]

In [21]:
def get_punct(text):
    punct = 0
    for t in text:
         if t.is_punct:
            punct += 1
    return punct

In [22]:
get_punct(word_counts.text_sentence[0])

10

In [23]:
print(word_counts.text_sentence[0])

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [24]:
word_counts['num_punct'] = word_counts.text_sentence.apply(get_punct)

In [27]:
def count_true_for(predicate):
    def counts(sentence):
        count = 0
        for token in sentence:
            if predicate(token):
                count += 1
        return count
    return counts

def matches(character):
    return lambda token: token == character

alpha_predicate = lambda token: token.is_alpha
digit_predicate = lambda token: token.is_digit
title_predicate = lambda token: token.is_title

In [28]:
word_counts['num_alpha'] = word_counts.text_sentence.apply(count_true_for(alpha_predicate))
word_counts['num_digit'] = word_counts.text_sentence.apply(count_true_for(digit_predicate))
word_counts['num_title'] = word_counts.text_sentence.apply(count_true_for(title_predicate))

In [29]:
word_counts.head()

,dreamy,warning,convenient,alteration,idiotic,ready,waiting,two,satisfied,tie,...,rational,earnest,snug,text_sentence,text_source,num_words,num_punct,num_alpha,num_digit,num_title
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67,10,57,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63,7,56,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33,4,29,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Oh, dear, !)",Carroll,3,1,2,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(I, shall, be, late, !, ')",Carroll,6,2,4,0,1


In [38]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [39]:
clf = SVC()
clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.7068965517241379

Test set score: 0.7138157894736842


### Challenge 1:

In [53]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [54]:
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
hamlet = re.sub(r'VOLUME \w+', '', hamlet)
hamlet = re.sub(r'CHAPTER \w+', '', hamlet)
hamlet = text_cleaner(hamlet)

In [55]:
# Parse our cleaned data.
hamlet_doc = nlp(hamlet)

In [56]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
hamlet_sents = [[sent, "Hamlet"] for sent in hamlet_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
hamlet_sents = pd.DataFrame(hamlet_sents + persuasion_sents)
hamlet_sents.head()

,0,1
0,"(Actus, Primus, .)",Hamlet
1,"(Scoena, Prima, .)",Hamlet
2,"(Enter, Barnardo, and, Francisco, two, Centine...",Hamlet
3,"(Barnardo, .)",Hamlet
4,"(Who, 's, there, ?)",Hamlet


In [57]:
# Set up the bags.
hamletwords = bag_of_words(hamlet_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(hamletwords + persuasionwords)

In [58]:
hamlet_bow = bow_features(hamlet_sents, common_words)
print('done')

[2019-02-10T18:57:16.301412] Processing row 0

[2019-02-10T19:00:28.141804] Processing row 50

[2019-02-10T19:04:55.551012] Processing row 100

[2019-02-10T19:09:28.737242] Processing row 150

[2019-02-10T19:15:27.021722] Processing row 200

[2019-02-10T19:20:40.267224] Processing row 250

[2019-02-10T19:24:16.774752] Processing row 300

[2019-02-10T19:26:45.018237] Processing row 350

[2019-02-10T19:32:28.867460] Processing row 400

[2019-02-10T19:36:04.553797] Processing row 450

[2019-02-10T19:39:40.335269] Processing row 500

[2019-02-10T19:43:52.656256] Processing row 550

[2019-02-10T19:47:49.531674] Processing row 600

[2019-02-10T19:51:03.913848] Processing row 650

[2019-02-10T19:55:12.449065] Processing row 700

[2019-02-10T19:58:50.334555] Processing row 750

[2019-02-10T20:03:09.445470] Processing row 800

[2019-02-10T20:07:24.984445] Processing row 850

[2019-02-10T20:10:50.569768] Processing row 900

[2019-02-10T20:13:35.733875] Processing row 950

[2019-02-10T20:16:59.56

In [59]:
hamlet_bow['num_punct'] = hamlet_bow.text_sentence.apply(get_punct)
hamlet_bow['num_alpha'] = hamlet_bow.text_sentence.apply(count_true_for(alpha_predicate))
hamlet_bow['num_digit'] = hamlet_bow.text_sentence.apply(count_true_for(digit_predicate))
hamlet_bow['num_title'] = hamlet_bow.text_sentence.apply(count_true_for(title_predicate))

In [60]:
Y = hamlet_bow['text_source']
X = np.array(hamlet_bow.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9957627118644068

Test set score: 0.914050822122571
